# Use own MajoranaMapping and Transpile circuit

In [1]:
# imports
from __future__ import annotations

from functools import lru_cache

import numpy as np

from qiskit.quantum_info.operators import Pauli

from qiskit_nature.second_q.mappers.fermionic_mapper import FermionicMapper

from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.mappers import JordanWignerMapper, ParityMapper, BravyiKitaevMapper
from qiskit_nature.second_q.circuit.library import UCCSD, HartreeFock
import numpy as np
from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import SLSQP
from qiskit.primitives import Estimator

from qiskit_algorithms import AdaptVQE
from qiskit_nature.second_q.algorithms import GroundStateEigensolver

from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

import time

In [156]:
# imports for majorana mapping calculation
%load_ext autoreload
%autoreload 2

from fermionic_mappings import bk_majoranas
from electronic_hamiltonian import quadratic_terms
from qiskit.quantum_info import PauliList
from tableau import spread_node, spread_node_slice, anticommutation_matrix
import random
from cost_functions import weight, quadratic_term_mean_weight
from annealing import anneal
import matplotlib.pyplot as plt

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Define problem with ansatz

In [157]:
driver_h2o = PySCFDriver(atom="O 0.0 0.0 0.0; H 0.757 0.586 0.0; H -0.757 0.586 0.0", basis="sto-3g")
problem_h2o = driver_h2o.run()

driver_lih = PySCFDriver(atom="Li 0 0 0; H 0 0 1.5", basis="sto-3g")
problem_lih = driver_lih.run()

driver_h2 = PySCFDriver(atom="H 0 0 0; H 0 0 0.735", basis="sto-3g")
problem_h2 = driver_h2.run()

## Build MajoranaMapper

In [158]:
"""The Majorana Mapper. """
n = 0
def set_n(new_n):
    global n 
    n = new_n

def obtain_n():
    return n
    

class MajoranaMapper(FermionicMapper):
    """The Majorana fermion-to-qubit mapping."""
    
    @classmethod
    @lru_cache(maxsize=32)
    def pauli_table(cls, register_length: int) -> list[tuple[Pauli, Pauli]]:
        # pylint: disable=unused-argument
        pauli_table = []

        N = obtain_n()
        print("Num qubits: ", N)

        # obtain majorana paulis for problem instance
        x, z ,_= bk_majoranas(N)
            
        x, z, energies, energy_opt = anneal(x, z, explore=spread_node, energy=quadratic_term_mean_weight, cooling_rate=0.99995)

        paulis = PauliList.from_symplectic(z, x)
        #print(paulis)
        
        for i in range(int(len(paulis)//2)):
            pauli_table.append((paulis[i], paulis[int(len(paulis)//2+i)]))

        #print(pauli_table)

        # PauliList has the phase information.
        # Here, phase is unnecessary, so the following removes phase.
        for pauli1, pauli2 in pauli_table:
            pauli1.phase = 0
            pauli2.phase = 0
        return pauli_table

## Add Mapping

In [159]:
mappers = {}
mappers["JW"] = JordanWignerMapper()
#mappers["Parity"] = ParityMapper()
mappers["Bravi-Kitaev"] = BravyiKitaevMapper()
mappers["Majorana"] = MajoranaMapper()

## Create ansatz and transpile to ibm_torino

In [160]:
service = QiskitRuntimeService()
# We define a specific backend
torino_backend = service.backend("ibm_torino")

seed = 43

In [161]:
def obtain_resources(problem):   
    for key in mappers.keys():
        print(f"Generating ansatz and transpiled ansatz for: {key}")
        if key == "Majorana":
            set_n(2*problem.num_spatial_orbitals)

        qubit_op = mappers[key].map(problem.second_q_ops()[0])
    
        ansatz = UCCSD(
            problem.num_spatial_orbitals,
            problem.num_particles,
            mappers[key],
            initial_state=HartreeFock(
                problem.num_spatial_orbitals,
                problem.num_particles,
                mappers[key],
            ),
        )

        print(f"\tANSATZ: Count-Ops: {ansatz.decompose().decompose().decompose().count_ops()} size: {ansatz.decompose().decompose().decompose().size()} depth = {ansatz.decompose().decompose().decompose().depth()}")

        # transpile ansatz
        pm = generate_preset_pass_manager(
            backend=torino_backend,
            optimization_level=3,
            seed_transpiler=seed,
            layout_method="sabre",
        )

        start = time.time()
        circuit_opt = pm.run(ansatz)
        end = time.time()
        elapsed_seconds = end - start
        circuit_opt.draw("mpl", fold=False, idle_wires=False)
        print(f"\tANSATZ transpilation took: {elapsed_seconds:.2f}")
        print(f"\tANSATZ transpiled: Count-Ops: {circuit_opt.decompose().decompose().decompose().count_ops()} size: {circuit_opt.decompose().decompose().decompose().size()} depth = {circuit_opt.decompose().decompose().decompose().depth()}\n\n")

In [162]:
# proof of concept
obtain_resources(problem_h2)

Generating ansatz and transpiled ansatz for: JW
	ANSATZ: Count-Ops: OrderedDict([('h', 80), ('cx', 56), ('sdg', 20), ('s', 20), ('rz', 12), ('u', 2)]) size: 190 depth = 97
	ANSATZ transpilation took: 0.24
	ANSATZ transpiled: Count-Ops: OrderedDict([('u3', 271), ('u', 97), ('cx', 41)]) size: 409 depth = 304


Generating ansatz and transpiled ansatz for: Bravi-Kitaev
	ANSATZ: Count-Ops: OrderedDict([('cx', 38), ('h', 36), ('sdg', 10), ('rz', 10), ('s', 10), ('u', 3), ('ry', 2)]) size: 109 depth = 77
	ANSATZ transpilation took: 0.17
	ANSATZ transpiled: Count-Ops: OrderedDict([('u3', 273), ('u', 85), ('cx', 39)]) size: 397 depth = 315


Generating ansatz and transpiled ansatz for: Majorana
Num qubits:  4
	ANSATZ: Count-Ops: OrderedDict([('cx', 34), ('h', 28), ('rz', 9), ('sdg', 4), ('s', 4), ('u', 1), ('rzz', 1), ('rzx', 1), ('rx', 1)]) size: 83 depth = 55
	ANSATZ transpilation took: 0.33
	ANSATZ transpiled: Count-Ops: OrderedDict([('u3', 254), ('u', 77), ('cx', 40)]) size: 371 depth = 278

In [ ]:
obtain_resources(problem_h2o)

Generating ansatz and transpiled ansatz for: JW
	ANSATZ: Count-Ops: OrderedDict([('cx', 14360), ('h', 7840), ('sdg', 1960), ('s', 1960), ('rz', 1000), ('u', 10)]) size: 27130 depth = 18014
	ANSATZ transpilation took: 94.44
	ANSATZ transpiled: Count-Ops: OrderedDict([('u3', 91821), ('u', 18331), ('cx', 14838)]) size: 124990 depth = 90377


Generating ansatz and transpiled ansatz for: Bravi-Kitaev
	ANSATZ: Count-Ops: OrderedDict([('cx', 13108), ('h', 11122), ('sdg', 2889), ('s', 2889), ('rz', 999), ('u', 6), ('ry', 1)]) size: 31014 depth = 16849
	ANSATZ transpilation took: 62.05
	ANSATZ transpiled: Count-Ops: OrderedDict([('u3', 128746), ('u', 20578), ('cx', 18338)]) size: 167662 depth = 110150


Generating ansatz and transpiled ansatz for: Majorana
Num qubits:  14
	ANSATZ: Count-Ops: OrderedDict([('cx', 12208), ('h', 9488), ('sdg', 2286), ('s', 2286), ('rz', 993), ('u', 4), ('rzx', 3), ('ry', 2), ('rzz', 1), ('rx', 1)]) size: 27272 depth = 14817


In [163]:
obtain_resources(problem_lih)

Generating ansatz and transpiled ansatz for: JW
	ANSATZ: Count-Ops: OrderedDict([('cx', 8064), ('h', 4992), ('sdg', 1248), ('s', 1248), ('rz', 640), ('u', 4)]) size: 16196 depth = 10315
	ANSATZ transpilation took: 50.24
	ANSATZ transpiled: Count-Ops: OrderedDict([('u3', 40623), ('u', 9643), ('cx', 7134)]) size: 57400 depth = 43343


Generating ansatz and transpiled ansatz for: Bravi-Kitaev
	ANSATZ: Count-Ops: OrderedDict([('cx', 8680), ('h', 7440), ('sdg', 1670), ('s', 1670), ('rz', 640), ('u', 2)]) size: 20102 depth = 11043
	ANSATZ transpilation took: 33.56
	ANSATZ transpiled: Count-Ops: OrderedDict([('u3', 75662), ('u', 12922), ('cx', 10828)]) size: 99412 depth = 66261


Generating ansatz and transpiled ansatz for: Majorana
Num qubits:  12
	ANSATZ: Count-Ops: OrderedDict([('cx', 6946), ('h', 5782), ('sdg', 1348), ('s', 1348), ('rz', 632), ('ry', 4), ('u', 2), ('rzx', 2), ('rxx', 1), ('rx', 1)]) size: 16066 depth = 8411
	ANSATZ transpilation took: 32.85
	ANSATZ transpiled: Count-Ops: 